# Data Analysis for Stock Simulation

This notebook compiles statistics on the performance of various trading strategies and compares overall performance relative to changes in asset price.

It includes the following steps:

- Loading the CSV files (`price_data.csv` and `agent_wealth.csv`) generated by the simulation.
- Computing the overall average agent wealth over time and comparing it to the asset price.
- Calculating the correlation between asset price changes and average wealth changes.
- Mapping agents to their strategy types (using round-robin assignment for the first 100 agents and classifying additional agents as arbitrage).
- Plotting average agent wealth over time for each strategy.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the CSV files generated by the simulation
price_data = pd.read_csv('Data/price_data.csv', index_col=0)
wealth_data = pd.read_csv('Data/agent_wealth.csv', index_col=0)

print("Price Data:")
print(price_data.head())

print("\nWealth Data (agents over ticks):")
print(wealth_data.head())

## Overall Asset Price vs. Average Agent Wealth

We compute the overall average wealth across agents at each tick and compare it to the asset price.

In [ ]:
# Compute average agent wealth per tick (across all agents)
avg_wealth_over_time = wealth_data.mean(axis=1)

# Plot asset price vs. average agent wealth over time
ticks = price_data.index.astype(int)

plt.figure(figsize=(12,6))
plt.plot(ticks, price_data['Price'], marker='o', label='Asset Price')
plt.plot(ticks, avg_wealth_over_time, marker='x', label='Average Agent Wealth')
plt.xlabel('Tick')
plt.ylabel('Value')
plt.title('Asset Price vs. Average Agent Wealth Over Time')
plt.legend()
plt.show()

# Calculate percentage change in asset price between ticks
price_data['PriceChangePct'] = price_data['Price'].pct_change()

# Calculate percentage change in average wealth
avg_wealth_change = avg_wealth_over_time.pct_change()

# Compute correlation (drop NaN values from percentage change computations)
correlation = np.corrcoef(price_data['PriceChangePct'].dropna(), avg_wealth_change.dropna())[0, 1]
print(f"Correlation between price change and average wealth change: {correlation:.4f}")

## Average Wealth by Trading Strategy

Agents were assigned to strategies as follows:

- **Agents 0–99:** Round-robin assigned to `TrendFollowing`, `MeanReversion`, `Momentum`, `Breakout`, and `ValueInvesting`.
- **Agents with ID >= 100:** Designated as `Arbitrage`.

This cell maps agents to their strategies and plots the average wealth over time for each strategy.

In [ ]:
# Set the total number of non-arbitrage agents as used in the simulation
num_agents = 100

# Function to map agent ID to strategy name
def get_strategy_name(agent_id):
    if agent_id < num_agents:
        strategies = ['TrendFollowing', 'MeanReversion', 'Momentum', 'Breakout', 'ValueInvesting']
        return strategies[agent_id % len(strategies)]
    else:
        return 'Arbitrage'

# Build a dictionary mapping each strategy to the list of agent IDs (from wealth_data columns)
strategy_agents = {}
for col in wealth_data.columns:
    agent_id = int(col)  # assuming columns are agent IDs
    strat = get_strategy_name(agent_id)
    if strat not in strategy_agents:
        strategy_agents[strat] = []
    strategy_agents[strat].append(col)

# Compute average wealth over time for each strategy
strategy_avg_wealth = {}
for strat, agent_list in strategy_agents.items():
    strategy_avg_wealth[strat] = wealth_data[agent_list].mean(axis=1)

# Plot average wealth over time by strategy
plt.figure(figsize=(12, 6))
for strat, wealth_series in strategy_avg_wealth.items():
    plt.plot(wealth_series.index, wealth_series, marker='o', label=f'{strat} Average Wealth')
plt.xlabel('Tick')
plt.ylabel('Wealth')
plt.title('Average Agent Wealth Over Time by Strategy')
plt.legend()
plt.show()

## Final Wealth Distribution by Strategy

Finally, we create a box plot to visualize the distribution of final wealth by strategy.

In [ ]:
# Extract final wealth for each agent from the last tick
final_wealth = wealth_data.iloc[-1]

# Create DataFrame for final wealth per agent along with their strategy
agent_performance = pd.DataFrame({
    'AgentID': [int(a) for a in wealth_data.columns],
    'FinalWealth': final_wealth.values,
    'Strategy': [get_strategy_name(int(a)) for a in wealth_data.columns]
})

# Plot a boxplot of final wealth by strategy
plt.figure(figsize=(10,6))
agent_performance.boxplot(column='FinalWealth', by='Strategy')
plt.title('Final Wealth Distribution by Trading Strategy')
plt.suptitle('')
plt.xlabel('Strategy')
plt.ylabel('Final Wealth')
plt.show()

# Display summary statistics by strategy
performance_stats = agent_performance.groupby('Strategy')['FinalWealth'].agg(['mean', 'median', 'std', 'min', 'max'])
print("\nPerformance Statistics by Strategy:")
print(performance_stats)

## Summary

This notebook has loaded the simulation data, computed overall performance metrics, and broken down performance by trading strategy.

You can further extend the analysis by exploring additional correlations, detailed trade logs, or other performance metrics as needed.